## FastAI the latest favourite of Data Scientist
In this series, I'll try to walk you through fastAI series with detailed explaination.

Also the notebook will have the techniques taught in each part of course ,applied to come competition.

Let's get started

## Introduction to Random Forest

In this notebook, i'll try to explain you the fastAI's lecture series first topic.

We'll cover importance of Randomforest and why they are widely used today in all the competitions

### Imports
Check the comments about what each import is used for

In [ ]:
!pip install fastai==0.7.0

In [ ]:
#It will automatically reload the latest module when you start again
%load_ext autoreload
%autoreload 2
#Used to display plots and graphs inside Jupyter
%matplotlib inline

In [ ]:
#The following are fastAi imports
from fastai.imports import * 
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

To check what each library do or it's source code just do the below thing
to find what it does put ?
to find it's source code put ??

In [ ]:
?display

In [ ]:
??display

### We'll be working with  Kaggle Blue Book Buldozers competiton so let's import the data

In [ ]:
import os
print(os.listdir("../input"))

  So here we have train.csv and test.csv

In [ ]:
PATH = "../input/train/"

In [ ]:
# ! says it is a bash command and not a jupyter command, and {} says it is a python variable
!ls {PATH}

Lets check the data using the head command(bash command)

In [ ]:
!head -n 5 ../input/train/Train.csv

Now let's use the data using pandas which help's us to better visualize the data and also perform various operations on it

In [ ]:
df_raw = pd.read_csv(PATH+ 'Train.csv', low_memory = False ,parse_dates=['saledate'])

To check what a function do you can simple do to that function and click shift+tab, for more details click shift + tab + tab

In [ ]:
df_raw

In [ ]:
#function to display all the data of the dataframe at one go
def display_all(df):
    with pd.option_context("display.max_rows", 1000):
        with pd.option_context("display.max_columns", 1000):
            display(df)

In [ ]:
display_all(df_raw.transpose())

If you take a look at the metric of the competition then it is, RMSLE (Root mean squared log error) so let's convert the Y i.e target variable into it's log based values

In [ ]:
df_raw.SalePrice  = np.log(df_raw.SalePrice)

### Inital Processing
Lets start with preprocessing, we'll check for datatypes, nan values ,outiers and many more things which will tell us a lot about data we have 

## Curse of dimensionality
“Curse of dimensionality” → theoreticians don’t like many dimensions (aka columns), but in practical use the more information the better because you don’t know what might be.

No free lunch theorem → In theory, no one type of model will work well for any kind of data set, which true for random data sets

Jeremy’s “free lunch theorem” → in practice, random forest is the best model for most data b/c most data is not random. A random-forest decision tree works for almost every structured-data problem.

Q: if we have too many dimensions/fields won’t we run into co-linearity problems?
Random forests have almost no co-linearity problems

With completely off-the-shelf tools, i.e., without any customization, Jeremy’s general purpose random forest analysis places around 100th place in Bulldozer competition (in the top 25%) of all submissions.

### Let's convert the all variables which is categorical into continious

Note : our model need all the inputs to be in continious form, so we have to explicity handle it 

Let's convert date into continious, like we can extract many things outof date, like year, day, was it a weekday etc

So in fastai there is a add_datepart function which finds many fields for us like month year week day, monthend start of month etc

In [ ]:
df_raw.saledate.head(5)

In [ ]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [ ]:
add_datepart(df_raw, 'saledate')
df_raw.saleYear.head(5)

There are other many categorical data, we can directly handle them using train_cats function of fastai, it creates categorical varialbe for everything that is a string, it'll map int to sting

In [ ]:
train_cats(df_raw) #behind scenes everything will be converted into numbers

In [ ]:
df_raw.UsageBand.cat.categories

In [ ]:
#the UsageBand is in wired order, so we'll convert into order High Med Low
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'],ordered = True ,inplace = True)

There are many missing values, which RandomForest cannot handle directly, so we'll handle it now

In [ ]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

We'll replace categories with numeric code ,handle missing continious values and split the dependent variable into a seperate variable

Everything is done by proc_df

In [ ]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [ ]:
nas #this is created by proc_df they are new columns along with there mean values

In [ ]:
#all columns
df.columns

In [ ]:
#now check everything is numeric
df.head()

## lets run the model
Now let's create our basic randomforest regressor with no hyper parameter tuning 

In [ ]:
m = RandomForestRegressor(n_jobs=-1) #njobs = -1 use all the resouces for running our model
m.fit(df, y)
m.score(df, y)

Above we trained the model on entire data set ,so there is a possiblity that it may have overfitted the data and so the predictions are high, so let's split the data into train and validation set and check how our basic model is working on it

In [ ]:
def split_vals(a,n) : return a[:n].copy(), a[n:].copy()

In [ ]:
n_valid = 12000 #same as kaggle's test size
n_trn = len(df) - n_valid
raw_train, raw_valid = split_vals(df, n_trn)
x_train, x_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)


In [ ]:
x_train.shape, y_train.shape, x_valid.shape

In [ ]:
#the following code will print the score
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x_train), y_train), rmse(m.predict(x_valid), y_valid), m.score(x_train, y_train), m.score(x_valid, y_valid)]
    if hasattr(m ,'obb_score_'): res.append(m.obb_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(df,y)
print_score(m)

### Let's modify and tune the model to improve the predictions

In [ ]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=30000)
x_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(x_train, y_train)
print_score(m)

 ### Let's build a Single tree estimator
 the n_estimator in RandomForestRegressor tells us number of trees to use

In [ ]:
#RandomForest randomizes the things, so to stop it we set Bootstrap = False
#n_estimator depicts that we are using 1 tree with max_depth of that tree 3
m = RandomForestRegressor(n_jobs=-1, n_estimators=1, max_depth=3, bootstrap=False)
%time m.fit(x_train, y_train)
print_score(m)

In [ ]:
draw_tree(m.estimators_[0], df_trn, precision=3)

Now like a single tree, let's create a forest this is called as
## Bagging
Decision tree methods are extremely flexible and easy to use, and when ensembled (using bagging or boosting) are the state of the art on many practical tasks
first, by looking at the individual trees that make them up, then by learning about “bagging”, the simple trick that lets a random forest be much more accurate than any individual tree.

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, max_depth=35)
%time m.fit(x_train, y_train)
print_score(m)

In [ ]:
#let's check how each tree is working differently and how's there predictions
preds = np.stack([t.predict(x_valid)for t in m.estimators_])
preds[:, 0], np.mean(preds[:, 0]), y_valid[0]

In [ ]:
preds.shape

In [ ]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis = 0)) for i in range(40)])

We won't add more tree's as the accuracy is not increasing and the graph is flattening

## Out of bag score
Now what we could do is, we can pass few rows as validation set which were not used ealier, this way we have different validation set for each tree, and then we can average them
Kind of like crossvalidation
So it is like every row will be outofbag sample given to the tree

So if we pass obb_socre to RandomForest then it'll show us that score

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40,oob_score=True)
%time m.fit(x_train, y_train)
print_score(m)

## Reducing Overfitting
### Subsampling

Overfitting does major damage to our predictions on test set
We can do subsampling to reduce overfitting by 

In [ ]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')
x_train, _ = split_vals(df_trn, n_trn)
y_train, _ = split_vals(y_trn, n_trn)

The basic idea is this: rather than limit the total amount of data that our model can access, let's instead limit it to a different random subset per tree. That way, given enough trees, the model can still see all the data, but for each individual tree it'll be just as fast as if we had cut down our dataset as before.


In [ ]:
set_rf_samples(20000)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40,oob_score=True)
%time m.fit(x_train, y_train)
print_score(m)

We revert to using a full bootstrap sample in order to show the impact of other over-fitting avoidance methods.


In [ ]:
reset_rf_samples()

In [ ]:
def dectree_max_depth(tree):
    children_left = tree.children_left
    children_right = tree.children_right
    
    def walk(node_id):
        if(children_left[node_id] != children_right[node_id]):
            left_max = 1 + walk(children_left[node_id])
            righ_max = 1 + walk(children_right)
            return max(left_max, right_max)
        else:
            return 1
        root_node_id = 0
        return walk(root_node_id)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40,oob_score=True)
%time m.fit(x_train, y_train)
print_score(m)

In [ ]:
t = m.estimators_[0].tree_

In [ ]:
t.max_depth

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40,oob_score=True, min_samples_leaf = 5)
%time m.fit(x_train, y_train)
print_score(m)

In [ ]:
t = m.estimators_[0].tree_
dectree_max_depth(t)

Another way to reduce over-fitting is to grow our trees less deeply. We do this by specifying (with min_samples_leaf) that we require some minimum number of rows in every leaf node. This has two benefits:

There are less decision rules for each leaf node; simpler models should generalize better
The predictions are made by averaging more rows in the leaf node, resulting in less volatility

We can also increase the amount of variation amongst the trees by not only use a sample of rows for each tree, but to also using a sample of columns for each split. We do this by specifying max_features, which is the proportion of features to randomly select from at each split.

None
0.5
'sqrt'
1, 3, 5, 10, 25, 100

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40,oob_score=True, min_samples_leaf = 5, max_features=0.5)
%time m.fit(x_train, y_train)
print_score(m)

## Interpreting Machine learning Model
### Understanding the Data
Now we'll understand the data deeper and quickly, other thing we'll learn is we'll handle large dataset

* For unstructed data try deep learning, for structed mostly go with randomforest as it works almost all the time.

In [ ]:
 set_rf_samples(50000)

In [ ]:
def get_preds(t): return t.predict(x_valid)
%time preds = np.stack(parallel_trees(m, get_preds))
np.mean(preds[:, 0]), np.std(preds[:,0])

In [ ]:
x = raw_valid.copy()
x['pred_std'] = np.std(preds, axis = 0)
x['pred'] = np.mean(preds, axis = 0)
x.Enclosure.value_counts().plot.barh()

In [ ]:
flds = ['Enclosure','pred', 'pred_std']
enc_summ = x[flds].groupby('Enclosure', as_index = False).mean()
enc_summ

In [ ]:
raw_valid.ProductSize.value_counts().plot.barh()

## Feature Importance
It's not normally enough to just to know that a model can make accurate predictions - we also want to know how it's makign predictions. he most important way to see this is with feature importance

In [ ]:
fi = rf_feat_importance(m, df_trn)
fi[:10]

In [ ]:
fi.plot('cols', 'imp', figsize = (10,6), legend = False)

In [ ]:
def plot_fi(fi) : return fi.plot('cols', 'imp', 'barh', figsize = (12,7), legend=False)

In [ ]:
plot_fi(fi[:30])

In [ ]:
to_keep = fi[fi.imp > 0.005].cols
len(to_keep)

In [ ]:
df_keep = df_trn[to_keep].copy()
x_train, x_valid = split_vals(df_keep, n_trn)

In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(x_train, y_train)
print_score(m)

In [ ]:
fi = rf_feat_importance(m, df_keep)
plot_fi(fi)

In [ ]:
df_raw.YearMade.head(5)
df_raw.Coupler_System.head(5)

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.scatter(df_raw['YearMade'], df_raw['SalePrice'])
plt.ylabel('SalePrice')
plt.xlabel('YearMade')

plt.subplot(1,2,2)
sns.stripplot(df_raw['Coupler_System'], df_raw['SalePrice'])

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.stripplot(df_raw['ProductSize'], df_raw['SalePrice'],order = ['Large', 'Medium', 'Small', 'Mini', 'Compact'])

plt.subplot(1,2,2)
sns.stripplot(df_raw['fiProductClassDesc'], df_raw['SalePrice'])


In [ ]:
plt.hist(df_raw['YearMade'], bins = 10)
plt.show()

In [ ]:
df_raw['Coupler_System'].value_counts().plot(kind='bar')

In [ ]:
df_raw['ProductSize'].value_counts().plot(kind='bar')

In [ ]:
reset_rf_samples()

In [ ]:
x_train, x_valid = split_vals(df_keep, n_trn)

In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(x_train, y_train)
print_score(m)

In [ ]:
set_rf_samples(50000)

## Removing redundant featire

In [ ]:
from scipy.cluster import hierarchy as hc

In [ ]:
corr = np.round(scipy.stats.spearmanr(df_keep).correlation, 4)
corr_condensed = hc.distance.squareform(1-corr)
z = hc.linkage(corr_condensed, method='average')
plt.figure(figsize=(15,15))
dendrogram = hc.dendrogram(z, labels=df_keep.columns, orientation='left', leaf_font_size = 16)
plt.show()

In [ ]:
def get_obb(df):
    m = RandomForestRegressor(n_estimators=30,min_samples_leaf=5, max_features=0.6, n_jobs=-1, oob_score=True)
    x, _ = split_vals(df, n_trn)
    m.fit(x, y_train)
    return m.oob_score_

In [ ]:
get_obb(df_keep)

In [ ]:
for c in ('saleYear', 'saleElapsed', 'fiModelDesc', 'fiBaseModel', 'Grouser_Tracks', 'Coupler_System'):
    print(c, get_obb(df_keep.drop(c, axis = 1)))

In [ ]:
to_drop = ['saleYear', 'fiBaseModel', 'Grouser_Tracks']
get_obb(df_keep.drop(to_drop, axis = 1))

In [ ]:
df_keep.drop(to_drop, axis = 1, inplace=True)
x_train, x_valid = split_vals(df_keep, n_trn)

In [ ]:
reset_rf_samples()
m = RandomForestRegressor(n_estimators=60,min_samples_leaf=5, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(x_train, y_train)
print_score(m)

## Partial Dependence
We can use ggplot to check the partial dependence

In [ ]:
df_raw.YearMade[df_raw.YearMade<1950] = 1950
df_keep['age'] = df_raw['age'] = df_raw.saleYear - df_raw.YearMade

In [ ]:
x_train, x_valid = split_vals(df_keep, n_trn)
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.6, n_jobs=-1)
m.fit(x_train, y_train)
plot_fi(rf_feat_importance(m, df_keep))

In [ ]:
m = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
m.fit(x_train, y_train)
print_score(m)

So we can see that the score are pretty awesome after feature importance, it would have probably landed us in top 10

## End
So this is the  notebook which explore the first 4 part of FastAI's machine learning course, we used here RandomForest as our model and trained the model on preprocessed data.

Stay tune for the next notebook which will cover next part of FastAI's series.

Upvote the kernel if you find it helpful

Connect on Linkedin - https://www.linkedin.com/in/savannahar/